In [2]:
#!/usr/bin/python3

from pyAudioAnalysis import audioBasicIO
from pyAudioAnalysis import audioFeatureExtraction
import matplotlib.pyplot as plt
import subprocess as subp
import os, time, sys
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib

def getWavFiles():
    files = subp.check_output(["ls"]).decode("utf8").split("\n")
    files = [f for f in files if f.split(".")[-1] == "wav"]
    return files

# Gets feature vector for a wav song
counter = 0
def getFeatureVector(file):
    global counter

    try:
        [Fs, x] = audioBasicIO.readAudioFile(file);
        x = audioBasicIO.stereo2mono(x)
    except:
        print("Error on file: ", file)
        return None

    # We cut the audio to a 30 second window in the middle of the song
    # If the audio is shorter than 30 second, we discard it
    nSamples = Fs * 30
    if isinstance(x, int):
        return None
    elif len(x) < nSamples:
        return None
    offset   = (len(x) - nSamples) // 2
    x = x[offset:offset+nSamples]

    print("{}\tSampling rate, number of samples: {} {}".format(counter, Fs, len(x)))
    counter += 1

    mterm, sterm, f_names = audioFeatureExtraction.mtFeatureExtraction(x, Fs, 1*Fs, 1*Fs, 0.050*Fs, 0.025*Fs);

    # Should return 68 mid-term features per mid-term window (30 windows)
    return mterm.ravel()

def getDataset():
    dataset = []

    os.chdir("music_negative/")
    files = getWavFiles()
    for f in files:
        features = getFeatureVector(f)
        if features is not None:
            dataset.append([features, 0])
    os.chdir("..")

    os.chdir("music_positive/")
    files = getWavFiles()
    for f in files:
        features = getFeatureVector(f)
        if features is not None:
            dataset.append([features, 1])
    os.chdir("..")

    return np.array(dataset)

def trainModel(trainData, solve="lbfgs", hidden=(10)):
    clf = MLPClassifier(solver=solve, alpha=1e-4, hidden_layer_sizes=hidden, max_iter=1000, activation="logistic")

    X = [ i[0] for i in trainData ]
    Y = [ i[1] for i in trainData ]
    clf.fit(X, Y)

    return clf

def experiment(dataset):
    # Create a randomized copy of the dataset
    rDataset = np.shuffle(dataset.copy())

    # k-fold cross-validation
    nFolds = 5
    foldSize = len(rDataset)//nFolds

    for solver in ["lbfgs", "adam", "sgd"]:
        print("{}\t{}\t{}\t{}\t{}".format("solver", "layrs", "acc", "train", "test"))

        for layers in [(1), (2), (3), (4), (5), (6), (7), (8), (9), (10), (20), (30), (40), (50), (60), (5,2), (5,4), (10,2), (10,4), (20,2), (20,4)]:
            success = 0
            attempts = 0

            execTries = 10
            testTime  = 0
            trainTime = 0

            for execId in range(execTries):
                for i in range(nFolds):
                    train_idx = np.array([ True for i in range(len(rDataset)) ])
                    train_idx[i*foldSize:(i+1)*foldSize] = False

                    if i == nFolds-1:
                        train_idx[-1] = False

                    test_idx = train_idx == False

                    train = rDataset[train_idx]
                    test  = rDataset[test_idx]

                    # train model
                    beg = time.time()
                    clf = trainModel(train, solver, layers)
                    trainTime += time.time() - beg

                    beg = time.time()

                    # Calculate accuracy
                    for [x,y] in test:
                        y_star = clf.predict([x])
                        # print("[{},{}]".format(y,y_star))
                        if y_star == y:
                            success += 1
                        attempts += 1

                    testTime += time.time() - beg

            print("{}\t{}\t{}\t{}\t{}".format(solver, layers, success / attempts, trainTime / execTries, testTime / execTries))

def zNormalize(dataset):
    # Each row in the dataset has the feature vector, then its class
    feats = dataset[:,0]

    # Transform to matrix
    matrix = np.matrix([[col for col in row] for row in feats])

    # For each feature (column), we z-normalize it
    for col in range(matrix.shape[1]):
        mean = np.mean(matrix[:,col])
        std  = np.std(matrix[:,col])
        matrix[:,col] = (matrix[:,col] - mean) / std

    # Replace features in the dataset
    for row in range(matrix.shape[0]):
        dataset[row,0] = np.array(matrix[row,:]).flatten()


if len(sys.argv) == 1:
    helpmsg = """Usage: {} [option [option...]]

OPTIONS
=======
    -f    - generate features
    -e    - run experiments using K-fold cross validation
""".format(sys.argv[0])

    print(helpmsg, end="")
    sys.exit(1)

# Get or generate dataset
if False:
    dataset = getDataset()
    zNormalize(dataset)
    np.save("features.npy", dataset)
else:
    try:
        dataset = np.load("features.npy")
    except:
        print("Please generate features first. Run the program without arguments for help.")
        sys.exit(2)
        
# Execute experiments
if False:
    experiment(dataset)

In [17]:
def getGenres():
    with open("genres.csv") as fp:
        genres = fp.read()
    
    gdict = {}
    genres = [ g.split(",") for g in genres.strip().split("\n") ]
    for row in genres:
        gdict[int(row[0])] = row[1]
    
    return gdict
    
genres = getGenres()
genres

{2: 'Hip-Hop',
 5: 'Hip-Hop',
 10: 'Pop',
 140: 'Folk',
 141: 'Folk',
 148: 'Experimental',
 182: 'Rock',
 190: 'Folk',
 193: 'Folk',
 194: 'Folk',
 197: 'Folk',
 200: 'Folk',
 203: 'Folk',
 204: 'Folk',
 207: 'Folk',
 210: 'Folk',
 211: 'Folk',
 212: 'Folk',
 213: 'Pop',
 255: 'Rock',
 256: 'Rock',
 368: 'Rock',
 424: 'Experimental',
 459: 'Rock',
 534: 'Folk',
 540: 'Folk',
 546: 'Folk',
 574: 'Rock',
 602: 'Folk',
 615: 'Experimental',
 620: 'Folk',
 621: 'Folk',
 625: 'Folk',
 666: 'International',
 667: 'International',
 676: 'Hip-Hop',
 690: 'Rock',
 694: 'Hip-Hop',
 695: 'Hip-Hop',
 704: 'International',
 705: 'International',
 706: 'International',
 707: 'International',
 708: 'International',
 709: 'International',
 714: 'Folk',
 715: 'Folk',
 716: 'Folk',
 718: 'Folk',
 777: 'Rock',
 814: 'Folk',
 821: 'Pop',
 822: 'Pop',
 825: 'Rock',
 853: 'International',
 890: 'Folk',
 892: 'Folk',
 897: 'Experimental',
 993: 'Rock',
 995: 'Experimental',
 997: 'Experimental',
 998: 'Expe

In [63]:
data = [ i[3] for i in dataset[:,0] ]
np.mean(data), np.std(data)

(-5.427757009278544e-16, 0.9999999999999999)